## **Import the necessary packages**

In [ ]:
# Import the necessary packages
import pandas as pd
import numpy as np
import os
import cv2
import shutil
import re

In [ ]:
# Checking the colab env
import tensorflow as tf
print(tf.test.gpu_device_name())
from tensorflow.python.client import device_lib
print("List of GPUs:", device_lib.list_local_devices())

##**Download the Dataset**

Just for initial experiments download the dataset into colab workspace and extract here.


In [ ]:
# Download the dataset or link dataset from GDrive
def download_external_dataset():
  if (os.path.isfile("Mini_Dataset.zip")):
    print("Dataset is already downloaded in workspace. Continuing with that...")
  else:
    !wget <LINK TO FILE>
    !unzip <ZIP FILE NAME> 

## **Connecting to Google Drive**

In [ ]:
# Connect to GDrive
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

In [ ]:
# Set the project path to "PATH" 
PATH = "<PROJECT FOLDER PATH>"
# Set the dataset path to "DATASET_PATH"
DATASET_PATH = "<DATASET FOLDER PATH>"



In [ ]:
%cd $PATH

## **Downloading and Configuring the Darknet**

Run this section for first time only. In later executions you can skip it.

In [ ]:
# Create a new folder for downloading Darknet
os.mkdir("Darknet")
!git clone 'https://github.com/AlexeyAB/darknet.git' os.path.join($PATH, "Darknet")

In [ ]:
!ls

Edit the "Makefile" and change the variables "GPU, CUDNN, OPENCV" to 1.

In [ ]:
# Define variables for paths of darknet and makefile
darknet_path = PATH + "/Darknet"
makefile_path = darknet_path + "/Makefile"
# Change the variables "GPU, CUDNN, OPENCV" to 1
with open(makefile_path, "r+") as makefile:
  makefile_contents = makefile.read()
  makefile_contents = re.sub("GPU=0", "GPU=1", makefile_contents)
  makefile_contents = re.sub("CUDNN=0", "CUDNN=1", makefile_contents)
  makefile_contents = re.sub("OPENCV=0", "OPENCV=1", makefile_contents)
  makefile.seek(0)
  makefile.write(makefile_contents)
  makefile.truncate()
  print("The variables 'GPU, CUDNN, OPENCV' are changed to 1.")

In [ ]:
# Change the directory to darknet
%cd $darknet_path
# Run the Makefile using 'make' command
!make

In [ ]:
# Check if darknet is properly installed or not
!./darknet
%cd $PATH

## **Create Training and Testing Data**

###**Create files data and names**

In [ ]:
# Method to create files data and class names
def create_files_data_and_names(dataset_path):
  """ Create classes.names """  
  # Counter for number of classes
  classes = 0
  # Creating file classes.names from existing classes.txt
  print("The following are the classes in out Dataset:\n")
  with open(dataset_path + '/' + 'classes.names', 'w') as names, open(dataset_path + '/' + 'classes.txt', 'r') as txt:
    for line in txt:
      names.write(line)
      print(line)
      classes += 1

  """ Create file labelled_data.data """
  with open(dataset_path + '/' + 'labelled_data.data', 'w') as data:
    print("\nThe file 'labelled_data.data' is saved in the following location:\n")
    print(os.path.join(dataset_path, "labelled_data.data"))
    data.write(f"classes = {classes} \n")
    # Location of the train.txt file
    data.write(f"train = {dataset_path}/train.txt\n")
    # Location of the test.txt file
    data.write(f"valid = {dataset_path}/test.txt\n")
    # Location of the classes.names file
    data.write(f"names = {dataset_path}/classes.names\n")
    # Location where to save weights
    data.write('backup = backup')

In [ ]:
# Creating the files data and names
create_files_data_and_names(DATASET_PATH)


###**Creating Train and Test txt files**
Now we have to create the Train and Test data with split of 70:30 ratio. It will create Train.txt with 70% of the files paths and Test.txt with 30% of the file paths. 30% test size is given by default. We can change it if we want to.

In [ ]:
def create_train_test_txt_files(dataset_path, test_size = 0.3):
  # Defining list to write paths in
  p = [dataset_path + "/" + file + "\n" for file in os.listdir(dataset_path) if (os.path.splitext(file)[1] == ".jpg")]

  # Slicing first test_size % of elements from the list to write into the test.txt file
  p_test = p[:int(len(p) * test_size)]
  # Deleting from initial list first test_size % of elements
  p = p[int(len(p) * test_size):]

  # Creating file train.txt and writing 85% of lines in it
  with open(dataset_path + '/train.txt', 'w') as train_txt:
      # Going through all elements of the list
      for e in p:
          # Writing current path at the end of the file
          train_txt.write(e)

  # Creating file test.txt and writing 15% of lines in it
  with open(dataset_path + '/test.txt', 'w') as test_txt:
      # Going through all elements of the list
      for e in p_test:
          # Writing current path at the end of the file
          test_txt.write(e)

  print(f"Train set files: {len(p)}")
  print(f"Test set files: {len(p_test)}")

In [ ]:
# Creating the train and test txt files
create_train_test_txt_files(DATASET_PATH, test_size = 0.3)

In [ ]:
 with open(DATASET_PATH + '/test.txt', 'r') as test_txt:
   lines = test_txt.readlines()
print(lines[0])

In [ ]:
len(os.listdir(DATASET_PATH))

Now create a folder 'Custom_Weights' in Project Folder. From here onwards we need to apply Transfer Learning.

In [ ]:
# Create 'Custom_Weights' and 'backup' folders
os.mkdir("Custom_Weights")
os.mkdir("backup")
print("'Custom_Weights' and 'backup' folders are created.")

In [ ]:
%cd "Custom_Weights"
# Download pre-trained DArknet weights
!wget https://pjreddie.com/media/files/darknet53.conv.74
%cd $PATH

In [ ]:
os.remove("Darknet/cfg/yolov3.cfg")
!unzip yolov3.zip
shutil.move('yolov3.cfg', "Darknet/cfg")

###**Configuring the 'yolov3.cfg' file.**
1. Goto Darknet Folder and under cfg folder we can find yolov3.cfg file
2. Open this file and change the following things.
    For Training:
        Uncomment the 3 lines (5, 6 ,7) and give the batch_size and subdivisions
    For Testing
        Uncomment the 3 lines (2, 3, 4) and give the batch_size and subdivisions
    We can change the 'maxbatches' in line 20 to (number of classes * 2000)
    We can change the 'steps' (min steps, max steps)
3. Now go to the last 3 YOLO Layers and change these yolo layers and preceding conv layers only.
    In first layer
        At line 603, change number of filters to (num of classes + 5) * 3
        At line 610, change number of classes to our num of classes
    Do the same thing for next 2 layers
        At line 689, 696 and 776, 783.
4. Save the file 

##**Start Training the Model**
Train the model with the given format

     !darknet/darknet detector train <labelled_data.data file path> <yolov3.cfg file path> <Custom Weights path> -gpus 0 -dont_show

In [ ]:
# Training the Model
!./Darknet/darknet detector train "Mini_Dataset/labelled_data.data" "Darknet/cfg/yolov3.cfg" "Custom_Weights/darknet53.conv.74" -gpus 0 -dont_show

##**Testing the Model**
Test the model with the given format on image

      !darknet/darknet detector test <labelled_data.data file path> <yolov3.cfg file path> <Trained Weights path> -thresh <Image Path>

In [ ]:
# Test the Model on image
!./Darknet/darknet detector test "./Dataset/labelled_data.data" "./Darknet/cfg/yolov3.cfg" "./backup/yolov3_last.weights" -thresh 0.20 "Image Path"

##**Testing the Model on Video**
Test the model with the given format on Video

      !darknet/darknet detector demo <labelled_data.data file path> <yolov3.cfg file path> <Trained Weights path> -thresh 0.20 -dont_show <Input Video Path> -out_filename <Output Video Path>

In [ ]:
# Test the Model on Video
!./Darknet/darknet detector demo "./Mini_Dataset/labelled_data.data" "./Darknet/cfg/yolov3.cfg" "./backup/yolov3_last.weights" -thresh 0.20 "./Video_IR/IR_AIRPLANE_001.mp4" -out_filename "./Video_IR/IR_AIRPLANE_001_out.mp4"


In [ ]:
# Resetting the environment for re-run
def reset_env():
  os.remove(DATASET_PATH + "/labelled_data.data")
  os.remove(DATASET_PATH + "/classes.names")
  os.remove(DATASET_PATH + "/train.txt")
  os.remove(DATASET_PATH + "/test.txt")
  os.rmdir(PATH + "/backup")
  os.rmdir(PATH +"/Custom_Weights")
  os.rmdir(PATH + "Darknet")
  print("Environment reset successfully")